In [4]:
%pip install gym
%pip install gym gym-retro
%pip install stable-baselines3
%pip install torch
%pip install importlib-metadata==4.13.0

  Using cached gym-0.26.2-py3-none-any.whl
  Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl.metadata (5.6 kB)
  Using cached cloudpickle-3.1.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached gym_notices-0.0.8-py3-none-any.whl.metadata (1.0 kB)
Using cached cloudpickle-3.1.0-py3-none-any.whl (22 kB)
Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl (19.8 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached gym_retro-0.8.0-cp38-cp38-macosx_10_13_x86_64.whl.metadata (3.6 kB)
  Using cached pyglet-1.5.29-py3-none-any.whl.metadata (7.6 kB)
Using cached gym_retro-0.8.0-cp38-cp38-macosx_10_13_x86_64.whl (146.1 MB)
Using cached pyglet-1.5.29-py3-none-any.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached stable_baselines3-2.3.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
  Using cached

In [5]:
import retro

ImportError: dlopen(/Users/suyogkhanal/Coding Projects/StreetFighterAI/.venv/lib/python3.8/site-packages/retro/_retro.cpython-38-darwin.so, 0x0002): tried: '/Users/suyogkhanal/Coding Projects/StreetFighterAI/.venv/lib/python3.8/site-packages/retro/_retro.cpython-38-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/suyogkhanal/Coding Projects/StreetFighterAI/.venv/lib/python3.8/site-packages/retro/_retro.cpython-38-darwin.so' (no such file), '/Users/suyogkhanal/Coding Projects/StreetFighterAI/.venv/lib/python3.8/site-packages/retro/_retro.cpython-38-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))